# Parallel Workflow for Essay Evaluation

In [60]:
# Parallel Workflow for Essay Evaluation
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import operator
import json
import os

In [61]:
load_dotenv()

True

In [62]:
# Get the Hugging Face token from the environment
hf_token = os.getenv("HUGGINGFACE_HUB_TOKEN")

# Check if token is loaded properly
if not hf_token:
    raise ValueError("HUGGINGFACE_HUB_TOKEN is not set in the environment!")

In [63]:
llm = HuggingFaceEndpoint(
    repo_id="moonshotai/Kimi-K2-Instruct",
    task="text-generation",
    huggingfacehub_api_token=hf_token,
)

In [64]:
# Wrap the LLM in a chat interface
model = ChatHuggingFace(llm=llm)

In [65]:
"""Use this while working with OpenAI/Antropic structured supported Models"""
# class EvaluationSchema(BaseModel):
#     feedback: str = Field(description="Detailed Feedback for the essay")
#     score: int = Field(description="Score out of 10", ge=0, le=10)

'Use this while working with OpenAI/Antropic structured supported Models'

In [66]:
# structured_model = model.with_structured_output(EvaluationSchema)

In [67]:
essay = """Artificial Intelligence (AI) is rapidly transforming the modern world, influencing industries, governance, and everyday life. From voice assistants and personalized recommendations to autonomous vehicles and predictive healthcare, AI is now embedded in the very fabric of human progress. As we advance into the Fourth Industrial Revolution, AI promises to redefine the way we work, learn, and interact with technology. Its potential to address critical challenges—such as climate change, resource management, and access to quality healthcare and education—marks it as a cornerstone of future development.

However, the future of AI also brings with it significant challenges. The fear of job displacement due to automation, increasing data privacy concerns, ethical dilemmas in algorithmic decision-making, and the growing digital divide between nations and communities highlight the complex risks associated with unchecked AI proliferation. Additionally, the use of AI in warfare, surveillance, and misinformation campaigns poses threats to democratic institutions and global security. Without robust regulatory frameworks, AI may amplify existing inequalities and create new forms of discrimination and dependence.

To ensure a balanced and beneficial future, a human-centric and ethical approach to AI is essential. Governments, academia, and industry must collaborate to build inclusive AI systems, prioritize transparency, and invest in public awareness and skill development. For a country like India, with its demographic advantage and growing digital infrastructure, AI offers an unprecedented opportunity to leapfrog into a knowledge-based economy. With proactive policymaking, responsible innovation, and a focus on equity, AI can become a force for social good and sustainable development."""


In [68]:
class EssayState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list, operator.add]
    avg_score: float


In [69]:
def parse_json_response(response_content):
    """Helper function to parse JSON from model response"""
    try:
        start = response_content.find('{')
        end = response_content.rfind('}') + 1
        json_str = response_content[start:end]
        return json.loads(json_str)
    except (json.JSONDecodeError, ValueError) as e:
        print(f"Failed to parse JSON: {e}")
        # Return default values if parsing fails
        return {"feedback": "Unable to generate feedback", "score": 0}

In [70]:
def evaluate_language(state: EssayState):
    prompt = f"""
    Evaluate the following essay for language and grammar. Provide detailed feedback and a score out of 10.

    Essay:
    {state['essay']}

    Respond in the following JSON format:
    {{
      "feedback": "<your feedback here>",
      "score": <score between 0 and 10>
    }}
    """
    
    response = model.invoke(prompt)
    data = parse_json_response(response.content)
    
    return {
        'language_feedback': data['feedback'],
        'individual_scores': [data['score']]
    }

def evaluate_analysis(state: EssayState):
    prompt = f"""
    Evaluate the following essay for analytical depth and argumentation. Provide detailed feedback and a score out of 10.

    Essay:
    {state['essay']}

    Respond in the following JSON format:
    {{
      "feedback": "<your feedback here>",
      "score": <score between 0 and 10>
    }}
    """
    
    response = model.invoke(prompt)
    data = parse_json_response(response.content)
    
    return {
        'analysis_feedback': data['feedback'],
        'individual_scores': [data['score']]
    }

def evaluate_clarity(state: EssayState):
    prompt = f"""
    Evaluate the following essay for clarity and structure. Provide detailed feedback and a score out of 10.

    Essay:
    {state['essay']}

    Respond in the following JSON format:
    {{
      "feedback": "<your feedback here>",
      "score": <score between 0 and 10>
    }}
    """
    
    response = model.invoke(prompt)
    data = parse_json_response(response.content)
    
    return {
        'clarity_feedback': data['feedback'],
        'individual_scores': [data['score']]
    }

def final_evaluation(state: EssayState)->EssayState:
    # Calculate the average score
    avg_score = sum(state['individual_scores']) / len(state['individual_scores'])
    
    # Compile overall feedback
    overall_feedback = (
        f"Language Feedback: {state['language_feedback']}\n\n"
        f"Analysis Feedback: {state['analysis_feedback']}\n\n"
        f"Clarity Feedback: {state['clarity_feedback']}\n\n"
        f"Individual Scores: {state['individual_scores']}\n"
        f"Average Score: {avg_score:.2f}/10"
    )
    
    return {
        'avg_score': avg_score,
        'overall_feedback': overall_feedback
    }

In [71]:

# Create the graph
graph = StateGraph(EssayState)

# Add nodes
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_clarity', evaluate_clarity)
graph.add_node('final_evaluation', final_evaluation)

# Add edges for parallel execution
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_clarity')

# All evaluation nodes feed into final evaluation
graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_clarity', 'final_evaluation')

graph.add_edge('final_evaluation', END)

In [72]:
# Compile and run the workflow
workflow = graph.compile()

# Initialize state with proper structure
initial_state = {
    'essay': essay,
    'language_feedback': '',
    'analysis_feedback': '',
    'clarity_feedback': '',
    'overall_feedback': '',
    'individual_scores': [],
    'avg_score': 0.0
}


    

In [73]:
# Run the workflow
try:
    final_state = workflow.invoke(initial_state)
    print("===  Essay Evaluation Results ===")
    print(final_state['overall_feedback'])
except Exception as e:
    print(f"Error running workflow: {e}")

===  Essay Evaluation Results ===
Language Feedback: Overall, the essay demonstrates excellent command of academic English. Sentence structures are varied and sophisticated, vocabulary is precise and contextually appropriate, and transitions between ideas are smooth. Minor points to note: (1) The phrase “embedded in the very fabric of human progress” is slightly clichéd; consider a fresher metaphor. (2) In the second paragraph, the long list of challenges could be broken into two sentences for better readability. (3) The final sentence is grammatically correct but slightly wordy; tightening it would enhance impact. No spelling, punctuation, or agreement errors were detected. The tone remains consistently formal and objective.

Analysis Feedback: The essay is well-structured and covers the key dimensions of the AI debate—promise, peril, and policy. However, it remains largely descriptive rather than analytical. Each paragraph lists phenomena (voice assistants, job loss, surveillance, et